In [258]:
# Load in the required dotenv vars
import os

# Config stuff
%reload_ext dotenv
%load_ext dotenv
%dotenv


UNSPLASH_ACCESS_KEY = os.environ.get("UNSPLASH_ACCESS_KEY")
UNSPLASH_ACCESS_KEY = UNSPLASH_ACCESS_KEY[1:-1]

WORKING_DIRECTORY = os.environ.get("WORKING_DIRECTORY")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# Get Ids to download
Get all ids from links.txt file

In [259]:
path = WORKING_DIRECTORY + "\\links.txt"

with open(path) as f:
    unsplash_pairs = f.readlines()

for i in range(len(unsplash_pairs)):
    unsplash_pairs[i] = unsplash_pairs[i].strip('\n')

for i in range(len(unsplash_pairs)):
    pieces = unsplash_pairs[i].split(' %=% ')
    unsplash_pairs[i] = (pieces[0], pieces[1], pieces[2].split(','))


print(unsplash_pairs)

[('gqsXXRPLeYc', 'StarFish', ['Animals', 'Ocean'])]


# Request from Unsplash

In [260]:
import pyunsplash

py_un = pyunsplash.PyUnsplash(api_key=UNSPLASH_ACCESS_KEY)

unsplash_photos = []

for unsplash_pair in unsplash_pairs:
    p = py_un.photos(type_="single", photo_id=unsplash_pair[0])

    print(p.entries.get_attribution(format='txt'))

    unsplash_photos.append(p)

shouldLog = False
if shouldLog:
    for unsplash_photo in unsplash_photos:
        print(unsplash_photo.entries.get_attribution(format='txt'))
        # print(unsplash_photo.entries.get_attribution(format='html'))

Photo by Taylor Beach on Unsplash


In [261]:
import requests

for unsplash_photo in unsplash_photos:
    response = requests.get(unsplash_photo.entries.link_download, allow_redirects=True)
    open(WORKING_DIRECTORY + '\\' + unsplash_photo.entries.id + '.jpg', 'wb').write(response.content)

In [262]:
from PIL import Image

shouldDisplay = False

max_display = 2
i = 0
while shouldDisplay and i < len(unsplash_photos) and i < max_display:
    display(Image.open(WORKING_DIRECTORY + '\\' + unsplash_photo.entries.id + '.jpg'))
    i += 1

# Create JSON object with Puzzle Metadata
To integrate with GeoSwapper

Sample JSON:
{ 
    "Questions": [
      {
        "author": "Kaleb Tapp",
        "url": "https://unsplash.com/photos/3mvzYkSF-qY",
        "puzzle": "GoatPortrait",
        "answer": "Goat",
        "tags": [
          "Animals"
        ]
      }
    ]
}

Example of json format
json = {"Questions": [{"author": "a", "url": "example.com", "puzzle": a, "answer": placeholder, "tags": ["TagA", "TagB"]}]}

In [263]:
import json
from os.path import exists

question_dictionary = {}

questions = []

for i in range(len(unsplash_photos)):
    obj = {}
    obj["author"] = unsplash_photos[i].entries.body["user"]["name"]
    obj["url"] = unsplash_photos[i].entries.body["links"]["html"]
    obj["puzzle"] = unsplash_pairs[i][0]
    obj["answer"] = unsplash_pairs[i][1]
    obj["tags"] = unsplash_pairs[i][2]
    # obj['tags'] = unsplash_photos[i].entries.body["tags"]

    questions.append(obj)

question_dictionary['Questions'] = questions

print(question_dictionary)

jsonString = json.dumps(question_dictionary)

path = WORKING_DIRECTORY + "\\MetaData0.json"
i = 0
while exists(path):
    path = path[:-6] + str(i) + ".json"
    i += 1

    print(path)

jsonFile = open(path,"w")
jsonFile.write(jsonString)
jsonFile.close()

{'Questions': [{'author': 'Taylor Beach', 'url': 'https://unsplash.com/photos/gqsXXRPLeYc', 'puzzle': 'gqsXXRPLeYc', 'answer': 'StarFish', 'tags': ['Animals', 'Ocean']}]}
